In [1]:
# Importing required libraries

import numpy as np

#from keras.preprocessing import image
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [4]:
train_datagen=ImageDataGenerator(rescale=1./255., preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('dog vs cat/dataset/train',
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=16,
                                                 class_mode='categorical')

Found 2002 images belonging to 2 classes.


In [5]:
valid_datagen=ImageDataGenerator(rescale=1./255., preprocessing_function=preprocess_input) #included in our dependencies

valid_generator=valid_datagen.flow_from_directory('dog vs cat/dataset/valid',
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=16,
                                                 class_mode='categorical')

Found 2000 images belonging to 2 classes.


## Standard CNN

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(2, activation="softmax"))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)             0         
__________

In [9]:
#compiling
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
model.fit_generator(
        train_generator,
        epochs=10,
        validation_data=valid_generator)

Epoch 1/10
126/126 [==============================] - 14s 108ms/step - loss: 0.6815 - acc: 0.5818 - val_loss: 0.6511 - val_acc: 0.6125
Epoch 2/10
126/126 [==============================] - 12s 98ms/step - loss: 0.6181 - acc: 0.6562 - val_loss: 0.5776 - val_acc: 0.6905
Epoch 3/10
126/126 [==============================] - 12s 98ms/step - loss: 0.5358 - acc: 0.7207 - val_loss: 0.5866 - val_acc: 0.6960
Epoch 4/10
126/126 [==============================] - 12s 97ms/step - loss: 0.4659 - acc: 0.7803 - val_loss: 0.5697 - val_acc: 0.7230
Epoch 5/10
126/126 [==============================] - 12s 99ms/step - loss: 0.3731 - acc: 0.8318 - val_loss: 0.6221 - val_acc: 0.7015
Epoch 6/10
126/126 [==============================] - 13s 100ms/step - loss: 0.2605 - acc: 0.8948 - val_loss: 0.8075 - val_acc: 0.7215
Epoch 7/10
126/126 [==============================] - 12s 99ms/step - loss: 0.1601 - acc: 0.9395 - val_loss: 0.8940 - val_acc: 0.7210
Epoch 8/10
126/126 [==============================] - 12s 97

## Transfer Learning

In [11]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(150,150,3))

In [12]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [13]:
model_tl = Sequential()
model_tl.add(base_model)
model_tl.add(Flatten())
model_tl.add(Dense(2, activation="softmax"))

In [14]:
model_tl.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 16386     
Total params: 14,731,074
Trainable params: 14,731,074
Non-trainable params: 0
_________________________________________________________________


In [16]:
#from keras import optimizers
#opt = optimizers.Adam(lr=0.0001)

In [17]:
#compiling
model_tl.compile(loss='categorical_crossentropy',optimizer="adam", metrics=['accuracy'])

In [18]:
model_tl.fit_generator(
        train_generator,
        epochs=10,
        validation_data=valid_generator)

Epoch 1/10
126/126 [==============================] - 13s 103ms/step - loss: 0.3372 - acc: 0.8437 - val_loss: 0.2819 - val_acc: 0.8840
Epoch 2/10
126/126 [==============================] - 12s 98ms/step - loss: 0.1983 - acc: 0.9186 - val_loss: 0.2294 - val_acc: 0.9020
Epoch 3/10
126/126 [==============================] - 12s 98ms/step - loss: 0.1243 - acc: 0.9598 - val_loss: 0.2395 - val_acc: 0.9010
Epoch 4/10
126/126 [==============================] - 12s 99ms/step - loss: 0.0905 - acc: 0.9742 - val_loss: 0.2364 - val_acc: 0.9010
Epoch 5/10
126/126 [==============================] - 12s 99ms/step - loss: 0.0700 - acc: 0.9807 - val_loss: 0.2618 - val_acc: 0.8890
Epoch 6/10
126/126 [==============================] - 12s 98ms/step - loss: 0.0573 - acc: 0.9896 - val_loss: 0.2440 - val_acc: 0.9015
Epoch 7/10
126/126 [==============================] - 12s 98ms/step - loss: 0.0399 - acc: 0.9960 - val_loss: 0.2526 - val_acc: 0.8990
Epoch 8/10
126/126 [==============================] - 12s 98m